In [1]:
import os

In [2]:
%pwd

'/home/gaurav/ds study/Study/Text-Summerization/notebooks'

In [3]:
os.chdir('../')

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int 
    per_device_eval_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int 
    save_steps: float
    gradient_accumulation_steps: int


In [5]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml,create_directories


In [6]:
# configurationManager
class ConfigurationManager:
    def __init__(self, config_file_path= CONFIG_FILE_PATH, param_file_path= PARAMS_FILE_PATH):
        
        self.config= read_yaml(config_file_path)
        self.param= read_yaml(param_file_path)

        if not os.path.exists(self.config.artifacts_root):
            create_directories(self.config.artifacts_root)

    def get_model_trainer_config(self) -> ModelTrainingConfig:
        config = self.config.model_trainer
        params= self.param.TrainingArguments
      
        if not os.path.exists(config.root_dir):
            create_directories([config.root_dir])
        
        model_trainer_config= ModelTrainingConfig(   
            root_dir =config.root_dir,
            data_path= config.data_path,
            model_ckpt= config.model_ckpt,
            num_train_epochs=params.num_train_epochs, 
            warmup_steps=params.warmup_steps,
            per_device_train_batch_size=params.per_device_train_batch_size, 
            per_device_eval_batch_size=params.per_device_eval_batch_size,
            weight_decay=params.weight_decay, 
            logging_steps=params.logging_steps,
            evaluation_strategy=params.evaluation_strategy, 
            eval_steps=params.eval_steps, 
            save_steps=params.save_steps,
            gradient_accumulation_steps=params.gradient_accumulation_steps
            )

        return model_trainer_config

In [7]:
from transformers import TrainingArguments,Trainer, DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer
from datasets import load_from_disk,load_dataset
import torch

/home/gaurav/study/anaconda3/envs/texts/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
class ModelTrainer:
    def __init__(self, config:ModelTrainingConfig) -> None:
        self.config= config

    def train(self):
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq2_data_collector = DataCollatorForSeq2Seq(tokenizer,model=model_pegasus)

        # load data
        data_samsum_pt = load_from_disk(self.config.data_path)

        # trainer_args = TrainingArguments(
        #     output_dir=self.config.root_dir,
        #     num_train_epochs=self.config.num_train_epochs, 
        #     warmup_steps=self.config.warmup_steps,
        #     per_device_train_batch_size=self.config.per_device_train_batch_size, 
        #     per_device_eval_batch_size=self.config.per_device_eval_batch_size,
        #     weight_decay=self.config.weight_decay, 
        #     logging_steps=self.config.logging_steps,
        #     evaluation_strategy=self.config.evaluation_strategy, 
        #     eval_steps=self.config.eval_steps, 
        #     save_steps=self.config.save_steps,
        #     gradient_accumulation_steps=self.config.gradient_accumulation_steps
        # )

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
            gradient_accumulation_steps=16
        ) 

        trainer = Trainer(model=model_pegasus,
                          args= trainer_args,
                          tokenizer=tokenizer,
                          data_collator=seq2seq2_data_collector,
                          train_dataset=data_samsum_pt['test'],
                          eval_dataset=data_samsum_pt['validation'])
        trainer.train()

        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, 'pegasus-samsum-model'))
        #save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,'tokenizer'))


In [15]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e



[2024-01-17 19:53:55,519: INFO: common: ymal file:config/config.yaml load successfully]
[2024-01-17 19:53:55,764: INFO: common: ymal file:params.yaml load successfully]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [21]:
config= ConfigurationManager()


[2024-01-17 19:19:51,216: INFO: common: ymal file:config/config.yaml load successfully]
[2024-01-17 19:19:51,224: INFO: common: ymal file:params.yaml load successfully]


In [22]:
config.config.model_trainer.root_dir

'artifacts/model_trainer'